In [48]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
pay = pd.read_csv(r"G:\My Drive\1. My Account\Palomar\Payment.CSV")

col = ['Prac Name', 'Loc Name', 'Post Date','E/I/A/B','Billed Amt','Pay Amt',
       'Source','Dt of Svc', 'CPT4','Payer Name', 'Fin Class','Rendering','Rsn Cds/Remarks']
filtered = pay[col]
filtered = filtered[filtered['Prac Name']!='Graybill Medical Group Inc']

amt = ['Billed Amt','Pay Amt']
filtered  = filtered.copy()
for a in amt:
    filtered[a] = filtered.loc[:,a].str.replace(
    '$', "", regex=False)\
        .str.replace('(', "-", regex=False)\
            .str.replace(')', "", regex=False)\
                .str.replace(',', "", regex=False)\
                    .str.replace(" ","", regex=False)
    

filtered['Billed Amt'] = filtered.loc[:,'Billed Amt'].astype(float)
filtered['Pay Amt'] = filtered.loc[:,'Pay Amt'].astype(float)

Dnl_Raw = filtered[(filtered.loc[:,'Loc Name']!='<Unknown>') & (filtered.loc[:,'Pay Amt'] == 0.00) 
                   & (filtered.loc[:,'Rsn Cds/Remarks']!="NaN") & (filtered.loc[:,'Source'] == 'Third party')]
Dnl_Raw = Dnl_Raw.copy()
Dnl_Raw['Post Date'] = pd.to_datetime(Dnl_Raw.loc[:,'Post Date'])
Dnl_Raw['Dt of Svc'] = pd.to_datetime(Dnl_Raw.loc[:,'Dt of Svc'])

Dnl_Raw = Dnl_Raw.copy()
Dnl_Raw.dropna(subset=['Rsn Cds/Remarks'], inplace=True)
Dnl_Raw["Index_rsn"] = np.arange(1, len(Dnl_Raw)+1, 1)

Dnl_Rsn_split=Dnl_Raw[["Index_rsn",'Rsn Cds/Remarks']]
split_columns = Dnl_Rsn_split['Rsn Cds/Remarks'].str.split(';', expand=True)
Dnl_Rsn_split = pd.concat([Dnl_Rsn_split[['Index_rsn']], split_columns], axis=1)

x=[0,1,2,3,4,5,6,7,8]
for i in x:
    Dnl_Rsn_split[i] = Dnl_Rsn_split[i].str.split(" ", expand=True)[0]

Dnl_Rsn_split =pd.melt(Dnl_Rsn_split,id_vars=['Index_rsn'],var_name='Rsn_code' ,value_name='Rsn' )
Dnl_Rsn_split.dropna(subset=['Rsn'], inplace=True)

cat = pd.read_excel(r"C:\Users\tinku.biswas\Downloads\SOP USMD Fresh Denials with Major Code Logic.xlsb", sheet_name='Denial Codes')
cat = cat[cat['Remark(if denial Yes/No)']=='Denial']

C:\Users\tinku.biswas\AppData\Local\Temp\ipykernel_14784\4063288783.py:4: DtypeWarning: Columns (20,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  pay = pd.read_csv(r"G:\My Drive\1. My Account\Palomar\Payment.CSV")


In [50]:
code=Dnl_Rsn_split.merge(cat[['Primary RSN Code','Remark(if denial Yes/No)']], left_on='Rsn', right_on='Primary RSN Code' )
code['sort_indx'] = code['Rsn'].apply(lambda x: 1 if isinstance(x,str) and 'CO' in x
                                      else 2 if isinstance(x, str) and 'OA' in x
                                      else 3 if isinstance(x, str) and 'PI' in x
                                      else 4 if isinstance(x, str) and 'PR' in x
                                      else 5)

code.sort_values(by=['Index_rsn','sort_indx'], inplace=True)
code['indicator'] = code.groupby(code['Index_rsn']).cumcount()
Major_code = code[code['indicator'] == 0]
Dnl_wt_code = Dnl_Raw.merge(Major_code[['Index_rsn','Rsn']], how='left', left_on='Index_rsn', right_on='Index_rsn')
Dnl_wt_code.dropna(subset=['Rsn'], inplace=True)
Dnl_wt_code.sort_values(by = ['E/I/A/B', 'Dt of Svc', 'CPT4', 'Post Date'], inplace = True)
Dnl_wt_code['First_denial_Ind'] = Dnl_wt_code.groupby(by=['E/I/A/B','Dt of Svc','CPT4','Post Date']).cumcount()
Dnl_wt_code['First_denial_Ind'] = Dnl_wt_code['First_denial_Ind'].apply(lambda x: 'First time Denial' if x == 0 else 'Repeat Denial')
Denial_final_raw = Dnl_wt_code.merge(cat[['Primary RSN Code', 'Category']], left_on='Rsn', right_on='Primary RSN Code')

In [ ]:
Denial_final_raw.pivot_table(
    index=[Denial_final_raw['Post Date'].dt.year,Denial_final_raw['Post Date'].dt.month],
    columns=['First_denial_Ind'],
    values=['Billed Amt','CPT4','E/I/A/B'],
    aggfunc= {'Billed Amt':'sum','CPT4':'count','E/I/A/B':'nunique'}
    )

In [54]:
Denial_2024 = Denial_final_raw[(Denial_final_raw['Post Date'].dt.year==2024)
                 &
                 (Denial_final_raw['First_denial_Ind']=='First time Denial')]

In [ ]:
Denial_2024.pivot_table(
    columns=[Denial_2024['Post Date'].dt.month],
    index=['Category'],
    values=['Billed Amt'],
    aggfunc= 'sum'
    )

In [ ]:
Denial_2024.pivot_table(
    columns=[Denial_2024['Post Date'].dt.month],
    index=['Category'],
    values=['E/I/A/B'],
    aggfunc= 'count'
    )

In [61]:
Denial_2024.pivot_table(
    columns=[Denial_2024['Post Date'].dt.month],
    index=['Prac Name', 'Loc Name'],
    values=['Billed Amt'],
    aggfunc= 'sum'
    )#.to_csv(r'C:\Users\tinku.biswas\Downloads\PHMG-Den.csv')

In [60]:
Denial_2024#.to_csv(r'C:\Users\tinku.biswas\Downloads\PHMG-Den_raw.csv')